In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain ragas datasets

In [74]:
import dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
dotenv.load_dotenv()
from langchain_community.vectorstores import Chroma
from ragas import evaluate# 0.0.22
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
)
from datasets import Dataset
from langchain_openai import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
import chromadb
from chromadb.config import Settings
import os

In [3]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL")
OPENAI_DEPLOYMENT = os.environ.get("OPENAI_DEPLOYMENT")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL")
EMBEDDING_DEPLOYMENT = os.environ.get("EMBEDDING_DEPLOYMENT")

In [77]:
chroma_client = chromadb.PersistentClient(path="vectordb")

# Evaluation part

### Ground truth

In [4]:
questions = [
    "Who was Alexei Navalny supposed to be exchanged for in the planned prisoner swap, according to his colleague Maria Pevchikh?",
    "What major issues with Boeing's safety culture were highlighted in the recent FAA report, and how is Boeing responding to the findings?",
    "What financial challenges is Europe facing in sustaining support for Ukraine, and how are officials suggesting addressing these challenges in the next 12 months?",
    "What is Sora AI, how does it work, and when can the general public expect to use it?",
    "What are the potential key issues that could impact the 2024 U.S. presidential election between Joe Biden and Donald Trump, and how might they impact the outcome?",
    "Why does Elon Musk want a bigger stake in Tesla, and what concerns does he express about the current structure of the company?",
    "What are the shopping habits of Gen Alpha, and why are mature brands interested in targeting this demographic?",
    "What is the main goal of Hamas in the Israel-Gaza conflict?",
    "What has Kim Jong Un recently declared about North Korea's relationship with South Korea, and what actions has he taken that raise concerns about potential military conflict?",
    "Why did Prince William pull out of the godfather's memorial service on 27.02.2024?"
]

ground_truths = [
    ["Alexei Navalny was supposed to be exchanged for Vadim Krasikov, a Russian hitman serving a life sentence for murder in Germany, as claimed by Maria Pevchikh, his colleague and chairwoman of his Anti-Corruption Foundation."],
    ["The FAA report on Boeing's safety culture found significant problems, including a 'disconnect' between senior management and employees, fear of retaliation when reporting safety concerns, confusion in safety reporting channels, and inadequate human factors in aviation safety. Boeing's response acknowledges the gaps, expressing commitment to learning from the findings, fostering a safety culture, and thoroughly reviewing the recommendations. The report identified 27 findings and 53 recommendations for improvement, which the FAA will review to ensure Boeing addresses them comprehensively."],
    ["Europe is grappling with the practicality of sustaining draining financial support for Ukraine amidst the deadlocked war and delayed aid. As spending on Ukraine faces political challenges, officials suggest exploring alternative funding sources, such as using frozen Russian assets to cover compensation costs. While some caution against setting a precedent, officials believe the EU's fundraising capabilities and potential arms deals, coupled with reduced reliance on the US, could help Europe continue supporting Ukraine in the coming year."],
    ["Sora AI is an artificial intelligence tool developed by OpenAI that can generate realistic videos up to 1 minute long based on textual prompts. It utilizes diffusion models, a method used in AI image generators, to create videos by understanding the association between images and accompanying alt text. As of now, Sora is not available to the general public; OpenAI is following a cautious approach by first testing it with a small group of 'red teamers' for potential harm or risks, followed by availability to visual artists, designers, and filmmakers before a potential public release, likely under a pay-to-use model similar to GPT. While Sora has shown significant advancements in AI video generation compared to previous attempts, it is not perfect, with occasional flaws and limitations seen in hand-picked videos released by OpenAI."],
    ["The potential key issues that could impact the 2024 U.S. presidential election between Joe Biden and Donald Trump include contrasting economic views, with Americans historically voting based on their economic well-being; the contentious topics of abortion and immigration, with Democrats focusing on abortion rights and Republicans emphasizing border security; and external factors like the Gaza War and its impact on Biden's support base. Other factors include crime rates, environmental concerns, and foreign policy. Additionally, the health and age of both candidates could be scrutinized, and the potential emergence of third-party candidates or independents might introduce unpredictability. Trump's legal challenges, facing 91 charges and four criminal trials, could influence public opinion, and the specter of the January 2021 Capitol attack may resonate differently among Republican and Democratic voters. Overall, these factors contribute to the complexity and uncertainty of the upcoming presidential race."],
    ["Elon Musk wants a bigger stake in Tesla to gain more control over the company's direction, especially regarding its investments in artificial intelligence (AI) features. He is concerned about Tesla's vulnerability to a 'takeover by dubious interests' and aims to have 25% voting control to ensure the company's leadership in AI and robotics. Musk suggests that without this control, he would prefer to develop products outside of Tesla."],
    ["Gen Alpha, born between 2010 and 2024, prefers to shop at adult brands like Lululemon, Sephora, Walmart, and Target, following the trends of their millennial parents. Adult brands are interested in targeting Gen Alpha due to their economic potential, with an estimated economic footprint of $5.46 trillion by 2029. These brands can secure the loyalty of the next generation by expanding their offerings to cater to the specific needs of Gen Alpha."],
    ["Hamas aims to create an Islamic state in place of Israel and rejects Israel's right to exist, justifying its attacks as responses to what it perceives as Israeli crimes against the Palestinian people. Additionally, Hamas seeks the release of Palestinian prisoners, an end to the blockade of the Gaza Strip, and the establishment of an independent Palestinian state."],
    ["Kim Jong Un recently declared that North Korea would no longer pursue reconciliation with South Korea, framing South Korea as the 'principal enemy.' He intensified nuclear threats, conducted missile tests, and abolished government agencies promoting cooperation and reunification. Analysts, including Robert L. Carlin and Siegfried Hecker, suggest he may be preparing for a military attack on South Korea."],
    ["Due to 'personal matter'."]
]
answers_llm = []
contexts_llm = [[""],[""],[""],[""],[""],[""],[""],[""],[""],[""]]

In [5]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION)
llm = AzureChatOpenAI(model_name=OPENAI_MODEL, azure_deployment=OPENAI_DEPLOYMENT,temperature=0)

In [7]:
def evaluation_llm(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
    )
    return result

In [21]:
def evaluation_rag(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  # most likely
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
        raise_exceptions=False,
    )
    return result

### General answers by LLM

In [11]:
template = """{question}"""
prompt = ChatPromptTemplate.from_template(template)

In [12]:
llm_chain =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm}
)


In [13]:
for query in questions:
    response = llm_chain.invoke({"question": query})
    answers_llm.append(response["response"].content)

Who was Alexei Navalny supposed to be exchanged for in the planned prisoner swap, according to his colleague Maria Pevchikh?
What major issues with Boeing's safety culture were highlighted in the recent FAA report, and how is Boeing responding to the findings?
What financial challenges is Europe facing in sustaining support for Ukraine, and how are officials suggesting addressing these challenges in the next 12 months?
What is Sora AI, how does it work, and when can the general public expect to use it?
What are the potential key issues that could impact the 2024 U.S. presidential election between Joe Biden and Donald Trump, and how might they impact the outcome?
Why does Elon Musk want a bigger stake in Tesla, and what concerns does he express about the current structure of the company?
What are the shopping habits of Gen Alpha, and why are mature brands interested in targeting this demographic?
What is the main goal of Hamas in the Israel-Gaza conflict?
What has Kim Jong Un recently d

In [14]:
llm_results = evaluation_llm(questions, answers_llm, contexts_llm, ground_truths)
print(llm_results)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 40/40 [00:11<00:00,  3.60it/s]


{'faithfulness': 0.9583, 'answer_relevancy': 0.4750, 'answer_similarity': 0.9093, 'answer_correctness': 0.5995}


### Naive RAG

In [60]:
loader = DirectoryLoader('../news', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [80]:
text_splitter = CharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db_naive = Chroma.from_documents(chunks, embeddings_client, persist_directory = "q&a/vectordb/naive")
retriever = db_naive.as_retriever()

Created a chunk of size 4589, which is longer than the specified 4000
Created a chunk of size 4082, which is longer than the specified 4000


In [81]:
template = """User input {question}. 
context {context}"""
prompt = ChatPromptTemplate.from_template(template)

In [82]:
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [92]:
answers_naive = []
contexts_naive = []
for query in questions:
    try:  
        response = retrieval_augmented_qa_chain.invoke({"question": query})
        # Access the response content
        answers_naive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_naive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_naive.append("No answer")
        context_full = retriever.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_naive.append(context_content)

In [93]:
result_naive_rag = evaluation_rag(questions, answers_naive, contexts_naive, ground_truths)
print(result_naive_rag)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:09<00:00,  6.65it/s]


{'faithfulness': 1.0000, 'answer_relevancy': 0.7608, 'context_precision': 0.9500, 'context_recall': 1.0000, 'answer_similarity': 0.9232, 'answer_correctness': 0.6148}


In [ ]:
# df = result.to_pandas()
# df

## try recursive text splitter

In [94]:
text_splitter = text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder()
chunks = text_splitter.split_documents(documents)
db = Chroma.from_documents(chunks, embeddings_client, persist_directory = "q&a/vectordb/recursive_basic")
retriever = db.as_retriever()

In [95]:
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [98]:
answers_recursive = []
contexts_recursive = []
for query in questions:
    try:  
        response = retrieval_augmented_qa_chain.invoke({"question": query})
        # Access the response content
        answers_recursive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_recursive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_recursive.append("No answer")
        context_full = retriever.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_recursive.append(context_content)

In [99]:
result_recursive = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
print(result_recursive)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.93it/s]


{'faithfulness': 1.0000, 'answer_relevancy': 0.5571, 'context_precision': 1.0000, 'context_recall': 1.0000, 'answer_similarity': 0.9134, 'answer_correctness': 0.6099}


## chunk size change

In [106]:
def change_chunk_size(db):
    retriever = db.as_retriever()
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
    )
    answers_recursive = []
    contexts_recursive = []

    for query in questions:
        try:  
            response = retrieval_augmented_qa_chain.invoke({"question": query})
            # Access the response content
            answers_recursive.append(response["response"].content)
            # Access the context content
            context_content = [context.page_content for context in response["context"]]
            contexts_recursive.append(context_content)  
        except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_recursive.append("No answer")
            context_full = retriever.get_relevant_documents(query)
            context_content = [context.page_content for context in context_full]
            contexts_recursive.append(context_content)


    result = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
    return result

In [107]:
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 100)
# chunks = text_splitter.split_documents(documents)
# print(len(chunks))
# db = Chroma.from_documents(chunks, embeddings_client, persist_directory = "q&a/vectordb/recursive_1000")
result_1000 = change_chunk_size(db)
print("CHUNK SIZE 1000")
print(result_1000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  22%|██▏       | 13/60 [00:02<00:07,  6.36it/s]Task exception was never retrieved
future: <Task finished name='Task-650' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py", line 2018, in aclose
    await self._transport.aclose()
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_transports\default.py", line 385, in aclose
    await self._pool.aclose()
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\s

CHUNK SIZE 1000
{'faithfulness': 0.9333, 'answer_relevancy': 0.7054, 'context_precision': 0.9500, 'context_recall': 0.9750, 'answer_similarity': 0.9271, 'answer_correctness': 0.6078}


In [108]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500, chunk_overlap = 50)
chunks = text_splitter.split_documents(documents)
print(len(chunks))
db = Chroma.from_documents(chunks, embeddings_client, persist_directory = "q&a/vectordb/recursive_500")
result_500 = change_chunk_size(db)
print("CHUNK SIZE 500")
print(result_500)

58


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.55it/s]


CHUNK SIZE 500
{'faithfulness': 0.9125, 'answer_relevancy': 0.8386, 'context_precision': 0.9583, 'context_recall': 1.0000, 'answer_similarity': 0.9290, 'answer_correctness': 0.6328}


In [110]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 2000, chunk_overlap = 200)
chunks = text_splitter.split_documents(documents)
print(len(chunks))
db = Chroma.from_documents(chunks, embeddings_client, persist_directory = "q&a/vectordb/recursive_2000")
result_2000 = change_chunk_size(db)
print("CHUNK SIZE 2000")
print(result_2000)

22


passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.56it/s]


CHUNK SIZE 2000
{'faithfulness': 0.9792, 'answer_relevancy': 0.7226, 'context_precision': 1.0000, 'context_recall': 1.0000, 'answer_similarity': 0.9125, 'answer_correctness': 0.6143}


### now time to look for different top-k